In [1]:
import os

os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/ra_joint_predictions')

In [2]:
from utils.config import Config

config = Config()

In [3]:
from dataset.joint_val_dataset import hands_joints_val_dataset, feet_joint_val_dataset, hands_wrists_val_dataset
from dataset.joints.joint_extractor import default_joint_extractor
from dataset.joints.joint_extractor_factory import get_joint_extractor

hands_ds = hands_joints_val_dataset(config, joint_extractor = get_joint_extractor('H', False), split_type = 'balanced', repeat = False)
feet_ds = feet_joint_val_dataset(config, joint_extractor = get_joint_extractor('F', True), split_type = 'balanced', repeat = False)

tf_hands_data = hands_ds.create_hands_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = False)
tf_hands_e_data = hands_ds.create_hands_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = True)
tf_feet_j_data = feet_ds.create_feet_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = False)
tf_feet_e_data = feet_ds.create_feet_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = True)

2020-06-27 12:02:19,486;WARNING - Missing permissions to create directory for caching!
2020-06-27 12:02:19,676;WARNING - Missing permissions to create directory for caching!
2020-06-27 12:02:19,950;WARNING - Missing permissions to create directory for caching!
2020-06-27 12:02:20,136;WARNING - Missing permissions to create directory for caching!
2020-06-27 12:02:20,322;WARNING - Missing permissions to create directory for caching!
[{'augment': <function random_flip at 0x7fa3d0002158>, 'p': 1}, {'augment': <function random_brightness_and_contrast at 0x7fa3d00021e0>}, {'augment': <function random_crop at 0x7fa3d00022f0>}, {'augment': <function random_gaussian_noise at 0x7fa3d0002378>, 'p': 0.2}, {'augment': <function random_rotation at 0x7fa3d0002268>}]
2020-06-27 12:02:25,234;WARNING - Missing permissions to create directory for caching!
2020-06-27 12:02:25,419;WARNING - Missing permissions to create directory for caching!
2020-06-27 12:02:25,604;WARNING - Missing permissions to create 

In [4]:
import numpy as np

from tensorflow import keras
from model.utils.metrics import mae_metric, rmse_metric, class_filter_rmse_metric, softmax_mae_metric, softmax_rmse_metric, class_filter_softmax_rmse_metric

def eval_model(label, dataset, model_paths, max_output, filter_model_paths = None, filter_cutoff = 0.5, rounding_cutoff = 0.3):
    pred_models = [keras.models.load_model(model_path, compile = False) for model_path in model_paths]
    
    if filter_model_paths is not None:
        filter_models = [keras.models.load_model(filter_model_path, compile = False) for filter_model_path in filter_model_paths]
    else:
        filter_models = None
    
    truths = []
    preds = []
    
    for x, y in dataset:
        if filter_models is not None:
            filter_y_pred = filter_models[0].predict(x)
            
            for filter_model in filter_models[1:]:
                filter_y_pred += filter_model.predict(x)
                
            filter_y_pred = filter_y_pred / len(filter_models)
            
            filter_y_pred = filter_y_pred > filter_cutoff
            filter_y_pred = filter_y_pred.astype(np.int32)
        else:
            filter_y_pred = np.ones(x.shape[0])
            
        y_pred = pred_models[0].predict(x)
        for pred_model in pred_models[1:]:
            y_pred += pred_model.predict(x)
            
        y_pred = y_pred / len(pred_models)
        
        y_pred = y_pred * filter_y_pred
        
        truths.extend(y.numpy()[:, 0])
        preds.extend(y_pred[:, 0])
    
    truths = np.array(truths)
    preds = np.array(preds)
    
    preds[preds < 0] = 0
    preds[preds > max_output] = max_output
    preds[preds <= rounding_cutoff] = 0.0
    
    non0_idx = np.where(truths != 0.0)[0]
    
    mae = np.mean(np.absolute(truths - preds))
    mae_std = np.std(np.absolute(truths - preds))
    mse = np.mean(np.square(truths - preds))
    mse_std = np.std(np.square(truths - preds))
    rmse = np.sqrt(mse)
    
    non0_rmse = np.sqrt(np.mean(np.square(truths[non0_idx] - preds[non0_idx])))
    non0_mae = np.mean(np.absolute(truths[non0_idx] - preds[non0_idx]))
    
    print(f'Model: {label} -- \nLoss: {mse} (+/- {mse_std}) \nMAE: {mae} (+/- {mae_std}) \nRMSE: {rmse} \nFilter0-RMSE: {non0_rmse} \nFilter0-MAE: {non0_mae}')

In [5]:
print('Hands Narrowing')

hand_j_filter_model_paths = ['../trained_models/narrowing/v7/hands_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5', 
                             '../trained_models/narrowing/v7/hands_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_test_gamma1.h5',
                             '../trained_models/narrowing/v7/narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6.h5']

Hands Narrowing


In [11]:
eval_model('3e-4 (25%, Current Model, 25%):', tf_hands_data[1], ['../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5'], 4, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.35)

Model: 3e-4 (25%, Current Model, 25%): -- 
Loss: 0.1420328215900876 (+/- 0.525037233699757) 
MAE: 0.12921246627893784 (+/- 0.35402960349129275) 
RMSE: 0.3768724208403788 
Filter0-RMSE: 0.7909338908760941 
Filter0-MAE: 0.5928418082473552


In [9]:
eval_model('3e-4 (25%, Small Dense):', tf_hands_data[1], ['../trained_models/narrowing/v8/hands_narrowing_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 4, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.35)

Model: 3e-4 (25%, Small Dense): -- 
Loss: 0.1451845731144687 (+/- 0.5833913253265093) 
MAE: 0.13562776830606846 (+/- 0.3560753874936937) 
RMSE: 0.38103093458992104 
Filter0-RMSE: 0.8060810699156774 
Filter0-MAE: 0.6034756832945664


In [10]:
eval_model('3e-4 (25%, Ensemble):', tf_hands_data[1], ['../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', '../trained_models/narrowing/v8/hands_narrowing_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 4, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.35)

Model: 3e-4 (25%, Ensemble): -- 
Loss: 0.12776453462403736 (+/- 0.4725189302287351) 
MAE: 0.12881788245705894 (+/- 0.33342238644595645) 
RMSE: 0.3574416520553213 
Filter0-RMSE: 0.7473862843074289 
Filter0-MAE: 0.5776081736636373


In [13]:
new_hand_j_model_paths = ['../trained_models/narrowing/v8/hands_narrowing_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5',
'../trained_models/narrowing/v8/hands_narrowing_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_gamma1_test.h5',
'../trained_models/narrowing/v8/narrowing_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5']

eval_model('3e-4 (25%, Ensemble, New Filter):', tf_hands_data[1], ['../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', '../trained_models/narrowing/v8/hands_narrowing_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 4, filter_model_paths = new_hand_j_model_paths, filter_cutoff = 0.3)

Model: 3e-4 (25%, Ensemble, New Filter): -- 
Loss: 0.1269735263256009 (+/- 0.46324654520846126) 
MAE: 0.13035717268129846 (+/- 0.3316331314210615) 
RMSE: 0.35633344822736035 
Filter0-RMSE: 0.7368427892951471 
Filter0-MAE: 0.5717693895629022


In [14]:
print('Feet Narrowing')

feet_j_filter_model_paths = ['../trained_models/narrowing/v7/feet_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5', 
                             '../trained_models/narrowing/v7/feet_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_test_gamma1.h5',
                             '../trained_models/narrowing/v7/narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6.h5']


eval_model('3e-4 (25%, Current Model):', tf_feet_j_data[1], ['../trained_models/narrowing/v7/feet_narrowing_joint_damage_model_complex_rewritten_64bs_125steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5'], 4, filter_model_paths = feet_j_filter_model_paths, filter_cutoff = 0.35)

Feet Narrowing
Model: 3e-4 (25%, Current Model): -- 
Loss: 0.2671964816926142 (+/- 0.9517533114333534) 
MAE: 0.1850603175086853 (+/- 0.48264807114107355) 
RMSE: 0.5169105161366077 
Filter0-RMSE: 1.1145134682852007 
Filter0-MAE: 0.8407738459984045


In [15]:
eval_model('3e-4 (25%, Small Dense):', tf_feet_j_data[1], ['../trained_models/narrowing/v8/feet_narrowing_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 4, filter_model_paths = feet_j_filter_model_paths, filter_cutoff = 0.35)

Model: 3e-4 (25%, Small Dense): -- 
Loss: 0.2836965940387474 (+/- 0.961666535574048) 
MAE: 0.19784777199392353 (+/- 0.494522854027787) 
RMSE: 0.5326317621384847 
Filter0-RMSE: 1.1367305974366224 
Filter0-MAE: 0.8960645077116203


In [16]:
eval_model('3e-4 (25%, Ensemble):', tf_feet_j_data[1], ['../trained_models/narrowing/v7/feet_narrowing_joint_damage_model_complex_rewritten_64bs_125steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', '../trained_models/narrowing/v8/feet_narrowing_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 4, filter_model_paths = feet_j_filter_model_paths, filter_cutoff = 0.35)

Model: 3e-4 (25%, Ensemble): -- 
Loss: 0.2604919103845064 (+/- 0.9150191495959892) 
MAE: 0.18852933780068443 (+/- 0.4742874647014634) 
RMSE: 0.5103840812412809 
Filter0-RMSE: 1.0984529643069059 
Filter0-MAE: 0.8528164263171051


In [14]:
new_feet_j_model_paths = ['../trained_models/narrowing/v8/feet_narrowing_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5',
'../trained_models/narrowing/v8/feet_narrowing_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_gamma1_test.h5',
'../trained_models/narrowing/v8/narrowing_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5']

eval_model('3e-4 (25%, Ensemble, New Filter):', tf_feet_j_data[1], ['../trained_models/narrowing/v7/feet_narrowing_joint_damage_model_complex_rewritten_64bs_125steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', '../trained_models/narrowing/v8/feet_narrowing_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 4, filter_model_paths = new_feet_j_model_paths, filter_cutoff = 0.35)

Model: 3e-4 (25%, Ensemble, New Filter): -- 
Loss: 0.262644676801499 (+/- 0.9101182127740578) 
MAE: 0.1955749683163978 (+/- 0.4737036083560468) 
RMSE: 0.5124887089502548 
Filter0-RMSE: 1.0727070508918661 
Filter0-MAE: 0.8268394644971917


In [18]:
print('Hands Erosion')

hand_e_filter_model_paths = ['../trained_models/erosion/v7/hands_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5', 
                             '../trained_models/erosion/v7/hands_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_test_gamma1.h5',
                             '../trained_models/erosion/v7/narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6.h5']

eval_model('3e-4 (25%, Current Model):', tf_hands_e_data[1], ['../trained_models/erosion/v7/hands_erosion_joint_damage_model_complex_rewritten_64bs_105steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5'], 5, filter_model_paths = hand_e_filter_model_paths, filter_cutoff = 0.3)

Hands Erosion
Model: 3e-4 (25%, Current Model): -- 
Loss: 0.37367892021692084 (+/- 1.8385369268971166) 
MAE: 0.17831631551732074 (+/- 0.5847069452616825) 
RMSE: 0.6112928268979776 
Filter0-RMSE: 1.9511561666130377 
Filter0-MAE: 1.5984274785892636


In [19]:
eval_model('3e-4 (25%, Small Dense):', tf_hands_e_data[1], ['../trained_models/erosion/v8/hands_erosion_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 5, filter_model_paths = hand_e_filter_model_paths, filter_cutoff = 0.3)

Model: 3e-4 (25%, Small Dense): -- 
Loss: 0.29702328358205893 (+/- 1.4160343665307393) 
MAE: 0.162314039681639 (+/- 0.520266697093218) 
RMSE: 0.5449984253023663 
Filter0-RMSE: 1.7398029088390217 
Filter0-MAE: 1.4194916679578669


In [20]:
eval_model('3e-4 (25%, Small Dense):', tf_hands_e_data[1], ['../trained_models/erosion/v7/hands_erosion_joint_damage_model_complex_rewritten_64bs_105steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', '../trained_models/erosion/v8/hands_erosion_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 5, filter_model_paths = hand_e_filter_model_paths, filter_cutoff = 0.3)

Model: 3e-4 (25%, Small Dense): -- 
Loss: 0.32104250265897555 (+/- 1.5299561128883403) 
MAE: 0.16866837993755446 (+/- 0.5409191069542805) 
RMSE: 0.5666061265632198 
Filter0-RMSE: 1.816842843835084 
Filter0-MAE: 1.4904338298669828


In [17]:
new_hands_e_filter_paths = ['../trained_models/erosion/v8/erosion_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5',
'../trained_models/erosion/v8/hands_erosion_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_gamma1_test.h5',
'../trained_models/erosion/v8/hands_erosion_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5']

eval_model('3e-4 (25%, Ensemble, New Filter):', tf_hands_e_data[1], ['../trained_models/erosion/v7/hands_erosion_joint_damage_model_complex_rewritten_64bs_105steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', '../trained_models/erosion/v8/hands_erosion_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 5, filter_model_paths = new_hands_e_filter_paths, filter_cutoff = 0.3)

Model: 3e-4 (25%, Ensemble, New Filter): -- 
Loss: 0.32301336658240964 (+/- 1.5310592048322391) 
MAE: 0.17373939601929633 (+/- 0.5411358321653259) 
RMSE: 0.5683426489208862 
Filter0-RMSE: 1.8100725724992295 
Filter0-MAE: 1.47402419646581


In [18]:
print('Feet Erosion')

feet_e_filter_model_paths = ['../trained_models/erosion/v7/feet_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5', 
                             '../trained_models/erosion/v7/feet_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_test_gamma1.h5',
                             '../trained_models/erosion/v7/narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6.h5']

eval_model('1e-3 (25%, Current Model):', tf_feet_e_data[1], ['../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_64bs_140steps_mixed_balanced_test_1e3.h5'], 10, filter_model_paths = feet_e_filter_model_paths, filter_cutoff = 0.3)

Feet Erosion
Model: 1e-3 (25%, Current Model): -- 
Loss: 0.9681167156109959 (+/- 4.9082364597777275) 
MAE: 0.3490601142138352 (+/- 0.9199313845477934) 
RMSE: 0.9839292228666633 
Filter0-RMSE: 2.2989376785946307 
Filter0-MAE: 1.701672994664737


In [19]:
eval_model('1e-3 (25%, Small Dense):', tf_feet_e_data[1], ['../trained_models/erosion/v8/feet_erosion_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 10, filter_model_paths = feet_e_filter_model_paths, filter_cutoff = 0.3)

Model: 1e-3 (25%, Small Dense): -- 
Loss: 0.8525335917904582 (+/- 4.0712127414041515) 
MAE: 0.34331820564064786 (+/- 0.8571267126079689) 
RMSE: 0.9233274564261902 
Filter0-RMSE: 2.0948709818279108 
Filter0-MAE: 1.5222559352586795


In [20]:
eval_model('1e-3 (25%, Ensemble):', tf_feet_e_data[1], ['../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_64bs_140steps_mixed_balanced_test_1e3.h5', '../trained_models/erosion/v8/feet_erosion_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 10, filter_model_paths = feet_e_filter_model_paths, filter_cutoff = 0.3)

Model: 1e-3 (25%, Ensemble): -- 
Loss: 0.8377317358112133 (+/- 4.057111282238705) 
MAE: 0.3378699711445487 (+/- 0.8506324813983975) 
RMSE: 0.9152768629279412 
Filter0-RMSE: 2.1197938313666804 
Filter0-MAE: 1.5586488506236633


In [21]:
new_feet_e_filter_paths = ['../trained_models/erosion/v8/erosion_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5',
'../trained_models/erosion/v8/feet_erosion_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_gamma1_test.h5',
'../trained_models/erosion/v8/feet_erosion_joint_damage_type_complex_gap_224x224_64bs_75epochs_adamW_3e4_1e6_5frozen_nodecay_fixed_bs_test.h5']

eval_model('1e-3 (25%, Ensemble, New Filter):', tf_feet_e_data[1], ['../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_64bs_140steps_mixed_balanced_test_1e3.h5', '../trained_models/erosion/v8/feet_erosion_small_bottlenecked_dense_1M_withNIH_nonfc_224x224_64bs_145steps_200epochs_adamW_3e4_1e6_50frozen_nodecay_fixed_bs_test.h5'], 10, filter_model_paths = new_feet_e_filter_paths, filter_cutoff = 0.35)

Model: 1e-3 (25%, Ensemble, New Filter): -- 
Loss: 0.8956644489549314 (+/- 4.479446037478796) 
MAE: 0.3497553825542167 (+/- 0.8793950314445066) 
RMSE: 0.9463955034524051 
Filter0-RMSE: 2.1999588040266627 
Filter0-MAE: 1.585042436595087
